In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from matplotlib import pyplot as plt
import seaborn
from pandas import DataFrame
import tqdm
from imblearn.under_sampling import RandomUnderSampler #アンダーサンプリング用
import pickle
# 機械学習用
from sklearn.cluster import KMeans #クラスタリング用
from sklearn.ensemble import RandomForestClassifier#ランダムフォレスト
from copy import deepcopy as cp
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression as LR

pd.set_option('display.width',400)#勝手に改行コードを入れられるのを防ぐ

### 前処理

In [3]:
#学習データの読み込み
result_df = pd.read_csv('train_asiya.csv')
result_df=result_df.drop(["Unnamed: 0"],axis=1)#csvファイルについている名無しの列を削除
#result_df=result_df.iloc[7000:]
result_df=result_df.drop(["racer_1_ID","racer_2_ID","racer_3_ID","racer_4_ID","racer_5_ID","racer_6_ID",],axis=1)#IDはいらないので削除
result_df=result_df.replace(0.0000,{"racer_1_ave_st_time":0.22})#新人のave_st_timeを0.22に
result_df=result_df.replace(0.0000,{"racer_2_ave_st_time":0.22})
result_df=result_df.replace(0.0000,{"racer_3_ave_st_time":0.22})
result_df=result_df.replace(0.0000,{"racer_4_ave_st_time":0.22})
result_df=result_df.replace(0.0000,{"racer_5_ave_st_time":0.22})
result_df=result_df.replace(0.0000,{"racer_6_ave_st_time":0.22})
result_df=result_df.replace(0.0000,{"racer_1_doub_win":0.02})#新人の着に絡む確率ave_st_timeを0.02に(新人の半期の偏差から導出)
result_df=result_df.replace(0.0000,{"racer_2_doub_win":0.02})
result_df=result_df.replace(0.0000,{"racer_3_doub_win":0.02})
result_df=result_df.replace(0.0000,{"racer_4_doub_win":0.02})
result_df=result_df.replace(0.0000,{"racer_5_doub_win":0.02})
result_df=result_df.replace(0.0000,{"racer_6_doub_win":0.02})

#レース番号ダミー化=======================================================
result_df_dummie=result_df
race_dummie_df=pd.get_dummies(result_df_dummie['number_race'])#number_raceをダミー化
for column, val in race_dummie_df.iteritems():
    result_df_dummie['race_{}'.format(int(column))]=val
result_df_dummie=result_df_dummie.drop('number_race',axis=1)

cols=list(result_df_dummie.columns)
male_cols=[s for s in cols if 'male' in s]#性別を示すカラムを取り出す

for col in male_cols:
    male_dummie_df=pd.get_dummies(result_df_dummie[col])#number_raceをダミー化
    for column, val in male_dummie_df.iteritems():
        result_df_dummie['{}_{}'.format(col,int(column))]=val
    result_df_dummie=result_df_dummie.drop('{}'.format(col),axis=1)

#ボート、モータもダミー化=============================================================================
cols=list(result_df_dummie.columns)
moter_cols=[s for s in cols if '_mo' in s]#モーター番号を示すカラムを取り出す
boat_cols=[s for s in cols if '_bo' in s]#ボート番号を示すカラムを取り出す
#boat もmoterも番号は1~99とする
numbers=np.arange(1, 100, 1)
empty_arr=[0]*len(result_df_dummie)
for col in moter_cols:
    for number in numbers:
        result_df_dummie['{}_{}'.format(col,int(number))]=empty_arr
    moter_dummie_df=pd.get_dummies(result_df_dummie[col])#モータ番号をダミー化
    for column, val in moter_dummie_df.iteritems():
        result_df_dummie['{}_{}'.format(col,int(column))]=val
    result_df_dummie=result_df_dummie.drop('{}'.format(col),axis=1)

#boat番号をダミー化
for col in boat_cols:
    for number in numbers:
        result_df_dummie['{}_{}'.format(col,int(number))]=empty_arr
    boat_dummie_df=pd.get_dummies(result_df_dummie[col])#boat番号をダミー化
    for column, val in boat_dummie_df.iteritems():
        result_df_dummie['{}_{}'.format(col,int(column))]=val
    result_df_dummie=result_df_dummie.drop('{}'.format(col),axis=1)
result_df=result_df_dummie

#クラスタリングラベルの付与==================================================================
#クラスタリングに使わないカラムを取り除く
train_drops_df=pd.DataFrame({})
drop_col_names=['result_com','money']
train_drops_df['result_com']=result_df['result_com']
train_drops_df['money']=result_df['money']
cluster_target_df=result_df.drop(drop_col_names,axis=1)

#クラスタリング
#分けてみるクラスタの数は[8,10]の2個
#cluster_target_df　　trainのデータからリザルトと配当金を取り除いたもの
target_num_cluster=[8,10]
#test_clustaring_df=train_has_PCA_df
clustaring_df=cluster_target_df
for num_cluster in target_num_cluster:
    pred = KMeans(n_clusters=num_cluster).fit_predict(cluster_target_df)
    clustaring_df['num={}'.format(num_cluster)]=pred


### 学習データのベースを作成(validとtrainに分割)

In [4]:
#result_comと配当金を戻す
clustaring_df['result_com']=result_df['result_com']#正解ラベルを戻してあげる
clustaring_df['money']=result_df['money']#配当金を戻してあげる
df=clustaring_df

#学習、テストデータ切り分け
num_data = len(df) 
num_valid = int(num_data/6.0)
valid_df = df.loc[(num_data-num_valid):num_data, :]#なるべく最新のデータを未知データに
train_df = df.loc[0:(num_data-num_valid-1), :]
train_money=pd.Series(train_df['money'])
valid_money=pd.Series(valid_df['money'])


#### モデルの性能評価用のデータフレーム

In [16]:
#出現数の分布
result_com_s=valid_df['result_com'].value_counts()
result_com_s=result_com_s.sort_index()
gain_mean=valid_df.groupby('result_com')['money'].mean()
gain_mean=gain_mean.sort_index()

gain_median=valid_df.groupby('result_com')['money'].median()
gain_median=gain_median.sort_index()
result_com_df=pd.DataFrame({'result_com':result_com_s.index,
                            'result_com_num':result_com_s.values,
                            'result_com_per':result_com_s.values/sum(result_com_s.values)*100,
                            'gain_mean':gain_mean.values,
                            'gain_median':gain_median.values,})
#result_com_df=result_com_df.set_index('result_com')

In [17]:
result_com_df

,result_com,result_com_num,result_com_per,gain_mean,gain_median
0,1.0,234,7.430930,1094.102564,785.0
1,2.0,204,6.478247,1280.196078,930.0
2,3.0,145,4.604636,2087.103448,1350.0
3,4.0,89,2.826294,2414.943820,1610.0
4,5.0,180,5.716100,1319.111111,950.0
...,...,...,...,...,...
112,115.0,2,0.063512,10300.000000,10300.0
113,116.0,2,0.063512,21955.000000,21955.0
114,117.0,3,0.095268,34193.333333,42250.0
115,118.0,3,0.095268,18846.666667,8670.0


## モデリング

### モデリング用＿動作確認

In [29]:
result_com=1
depth=5
model_score_df=pd.DataFrame(columns=['target_com','depth','target_per','総収益', '投資金額','出現数','購買予測数','利益率','購買的中率','的中数'])

#result_comごとの閾値の決定========================================================================
gain_th=10#利益率の閾値
result_s=result_com_df[result_com_df['result_com']==result_com]
buy_accuracy_th=result_s['result_com_per'].values[0]*1.1#買ったうちの的中率の閾値
num_tp_th=result_s['result_com_num'].values[0]*0.2#あたった回数の閾値(出現回数の20%が的中)
#===============================================================================
#学習データのラベル変換==========================================================
result_train_df=train_df
result_arr=[0]*len(result_train_df)
i=0
for result in result_train_df['result_com']:
    if ((result==result_com)):
        result_arr[i]=1
    else:
        result_arr[i]=0
    i+=1
result_train_df['result_com']=result_arr

result_valid_df=valid_df
result_arr=[0]*len(result_valid_df)
i=0
for result in result_valid_df['result_com']:
    if ((result==result_com)):
        result_arr[i]=1
    else:
        result_arr[i]=0
    i+=1

result_valid_df['result_com']=result_arr

result_train_df['money']=train_money
result_valid_df['money']=valid_money
#学習データラベル変換終わり============================================

for_arr=np.arange(1,100,1)
accuracy_arr=[0]*len(for_arr)
target_per_arr=[0]*len(for_arr)
pred_0=[0]*len(for_arr)
gain_arr=[0]*len(for_arr)
model_gain_arr=[0]*len(result_valid_df)
valid_gain_arr=valid_money.values
for sum_target_per in for_arr:
    
    index=sum_target_per-1
    target_per=50+sum_target_per
    target_per_arr[index]=target_per
    
    #モデルの評価指標値を格納するseries======================
    model_score_s=pd.Series(index=['target_com','depth','target_per','総収益', '投資金額','出現数','購買予測数','利益率','購買的中率','的中数'])
    model_score_s['target']=result_com#目標としているresult_comラベル番号
    model_score_s['depth']=depth#ハイパーパラメータ＿木の深さ
    model_score_s['target_per']=target_per#学習データ_1に対してどの程度の0のデータを持たせるか。
    #======================
    #trainの[0]に対して、target_perの割合の量[1]を持った学習データの作成
    # 一層目の判別機のtrainデータ　:terget_result_df
    target_df=result_train_df#ベースのデータフレームをコピー
    target_df=target_df.sample(frac=1, random_state=1)#シャッフル、時系列の偏りを無くす
    target_1_df=target_df[target_df['result_com']==1]
    len_1=len(target_1_df)
    target_0_df=target_df[target_df['result_com']==0]
    len_0=len(target_0_df)
    target_0_df=target_0_df.iloc[(len_0-int(len_1*(target_per/100))):len_0]#1に対する目標の割合ぶん0の結果だったレースを抽出（後ろから抽出）
    target_train_df=pd.concat([target_1_df, target_0_df])
    #以下学習パート======================================================
    target_x_train=target_train_df.drop('money',axis=1)
    target_x_train=target_x_train.drop('result_com',axis=1)
    target_y_train=target_train_df['result_com']
    #テストデータ
    target_y_valid=result_valid_df['result_com']
    target_x_valid=result_valid_df.drop('money',axis=1)
    target_x_valid=target_x_valid.drop('result_com',axis=1)
    RF = RandomForestClassifier(random_state=1,n_estimators=1000,max_depth=4)
    RF = RF.fit(target_x_train,target_y_train)


    #以下精度検証(１の正答率のみ調査)

    # 未知データに対する予測値
    predict_y_valid = RF.predict(target_x_valid)

    #[1]の正答率を見る
    pred_valid_df=pd.DataFrame({'pred':predict_y_valid
                              , 'valid':target_y_valid})
    num_1=len(pred_valid_df[pred_valid_df['valid']==1])
    count=0
    #追加　配当金の情報も考慮する。
    gain_index=0
    model_gain_arr=[0]*len(result_valid_df)
    for _, s in pred_valid_df.iterrows():
        if ((s['pred']==1) and (s['valid']==1)):
            count+=1#的中回数
            model_gain_arr[gain_index]=valid_gain_arr[gain_index]
        gain_index+=1
    #print('test accyracy: {}'.format((count/num_1)*100))
    gain_arr[index]=sum(model_gain_arr)
    accuracy_arr[index]=(count/num_1)*100
    try:
        pred_0[index]=pred_valid_df['pred'].value_counts()[0]
    except:
        pred_0[index]=0
    #scoreのseriesに情報書き込み==================
    model_score_s['総収益']=sum(model_gain_arr)
    model_score_s['投資金額']=100*sum(predict_y_valid)
    model_score_s['出現数']=sum(target_y_valid)
    model_score_s['購買予測数']=sum(predict_y_valid)
    model_score_s['利益率']=(sum(model_gain_arr)/100*sum(predict_y_valid))*100
    model_score_s['購買的中率']=(count/sum(predict_y_valid))*100
    model_score_s['的中数']=count
    
    #もしモデル作成したモデルが

c:\users\nabe0\desktop\nabepy\env\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\nabe0\desktop\nabepy\env\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\nabe0\desktop\nabepy\env\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http:

KeyboardInterrupt: 

In [24]:
type(result_s)

pandas.core.frame.DataFrame

In [20]:
result_s

,result_com,result_com_num,result_com_per,gain_mean,gain_median
0,1.0,234,7.43093,1094.102564,785.0
